# LLM Evaluation - La Victoria Lab

Bienvenido al **Large Language Models Evaluation** de La Victoria Lab. Esta notebook, nos muestra como evaluar el prompting de nuestro LLM y un sistema RAG, utilizando el framework promptflow. Utilizaremos el servicio Azure OpenAI.

## Índice

1. [Configuración Inicial](#Configuración-Inicial)
2. [Evaluación](#Evaluación)

## Configuración Inicial

Antes de comenzar, asegúrate de tener las librerías necesarias instaladas y configura las variables de entorno de manera segura utilizando un archivo `.env`.

### Instalación de Dependencias

Ejecuta la siguiente celda para instalar las librerías requeridas:

In [1]:
%%capture
%pip install promptflow
%pip install promptflow-evals
%pip install promptflow-tools
%pip install pandas

### Configuración de Variables de Entorno

In [2]:
import os

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://saltov2ibkindopenai.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "f8b4dbde4448455f9564c6aae5ddb211"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-15-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt4omini"

## Evaluación con Promptflow


In [3]:
from promptflow.core import AzureOpenAIModelConfiguration
from promptflow.evals.evaluators import GroundednessEvaluator, RelevanceEvaluator, CoherenceEvaluator, SimilarityEvaluator, F1ScoreEvaluator
from promptflow.evals.evaluate import evaluate
from azure.identity import DefaultAzureCredential
import pandas as pd
import json

In [4]:
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
)

In [6]:
relevance_eval = RelevanceEvaluator(model_config)
groundedness_eval = GroundednessEvaluator(model_config)
coherence_eval = CoherenceEvaluator(model_config)
similarity_eval = SimilarityEvaluator(model_config)
#f1score_eval = F1ScoreEvaluator(model_config)

In [7]:
question="quiero saber cuantos infiernos tiene la divina comedia?"
answer="En La Divina Comedia, el infierno está dividido en nueve círculos. Cada círculo representa un pecado diferente y un castigo correspondiente."
context="infierno que   merece: tantas veces se ciñe con la cola,   cuantos grados él quiere que sea echada Siempre delante de él se encuentran muchos; van esperando   cada uno su juicio,   hablan y escuchan, después las arrojan «Oh tú que vienes al doloso albergue me dijo Minos en cuanto   me vio, dejando el acto de tan alto oficio ;       26. Vi a Pantas..."
ground_truth = "En la obra los pajaritos vuelan, el cielo esta arriba. Muchas gracias por tu pregunta, tienes alguna consulta adicional?"

In [8]:
relevance_score = relevance_eval(
    question=question,
    answer=answer,
    context=context
)

groundedness_score = groundedness_eval(
    answer=answer,
    context=context

)

coherence_score = coherence_eval(
    question=question,
    answer=answer,
)

# similarity_score = similarity_eval(
#     question=question,
#     answer= answer,
#     ground_truth=ground_truth
# )

#print(relevance_score)
#print(groundedness_score)
print(coherence_score)
#print(similarity_score)

{'gpt_coherence': 5.0}


In [10]:
result = evaluate(
    data="conversation.jsonl",
    evaluators={
        #"relevance": relevance_eval,
        #"groundedness": groundedness_eval,
        "coherence": coherence_eval,
        #"similarity": similarity_eval
    },
    output_path="evals_results.json"
)

df_rows = pd.DataFrame(result['rows'])
display(df_rows)

[2024-10-30 17:03:21 -0500][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-10-30 17:03:22 -0500][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run promptflow_evals_evaluators_coherence_coherence_asynccoherenceevaluator_col9o025_20241030_170321_813073, log path: C:\Users\b44082\.promptflow\.runs\promptflow_evals_evaluators_coherence_coherence_asynccoherenceevaluator_col9o025_20241030_170321_813073\logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=promptflow_evals_evaluators_coherence_coherence_asynccoherenceevaluator_col9o025_20241030_170321_813073


[2024-10-30 17:03:50 -0500][promptflow.evals.evaluate._utils][ERROR] - Unable to log traces as trace destination was not defined.


2024-10-30 17:03:22 -0500   20676 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-10-30 17:03:22 -0500   20676 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2024-10-30 17:03:22 -0500   20676 execution.bulk     INFO     The timeout for the batch run is 3600 seconds.
2024-10-30 17:03:22 -0500   20676 execution.bulk     INFO     Current system's available memory is 6221.01953125MB, memory consumption of current process is 205.046875MB, estimated available worker count is 6221.01953125/205.046875 = 30
2024-10-30 17:03:22 -0500   20676 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 15, 'estimated_worker_count_based_on_memory_usage': 30}.
2024-10-30 17:03:32 -0500   20676 execution.bulk

,inputs.question,inputs.answer,outputs.coherence.gpt_coherence
0,Hola. Deseo información acerca de las tarjetas...,¡Hola! Soy tu asistente de préstamos de Interb...,3
1,Si,"¡Genial! Para empezar, por favor, me podrías p...",1
2,Hola deseo información sobre tarjetas de crédito,Entiendo que estás interesado en tarjetas de c...,5
3,Continuar con las pregyntas,Continuar con las preguntas,1
4,¡Perfecto! Empecemos con la primera pregunta:\...,¡Perfecto! Comencemos con la primera pregunta....,1
5,26,¡Gracias! Tienes 26 años. \n\nAhora vamos con ...,1
6,Maestría,¡Excelente! Has llegado hasta el nivel de maes...,2
7,Dos,¡Muy bien! Tienes dos títulos profesionales. \...,1
8,2023,¡Perfecto! Terminaste tu primera carrera en 20...,2
9,Mixta,¡Gracias por compartirlo! Tienes una situación...,1
